In [5]:
import sys
sys.path.append('./package/')

import networkx as nx

from package.utils import read_items, getItemsPrice
from package.topic import TopicModel
from package.itemset import ItemRelation, ItemsetFlyweight
from package.model import DiffusionModel
from package.algorithm import Algorithm

ITEMS_FILE = r"D:\\論文實驗\\data\\amazon\\sample_items.csv"
items = read_items(ITEMS_FILE)
for key, attr in items.items():
    if "price" not in attr:
        del items[key]
        
topicModel = TopicModel(3)
topicModel.randomTopic(items_id=items.keys())

graph = nx.diamond_graph()

relation = ItemRelation()
relation.construct(ITEMS_FILE)
itemset = ItemsetFlyweight(getItemsPrice(ITEMS_FILE), topicModel, relation)

model = DiffusionModel(graph, itemset, threshold=10**(-5))
algo = Algorithm(model, 10, depth=5)
# candidatedCoupons = algo.genFullItemCoupons(starThreshold=min(itemset.PRICE.values()), step=5, discountPercentage=[0.2, 0.4, 0.6, 0.8])
# candidatedCoupons = algo.genDiscountItemCoupons([0.2])
candidatedCoupons = algo.genAllCoupons(5)
print(len(candidatedCoupons))


720


In [6]:
from itertools import combinations
num_coupons = 2
couponsPowerset = []
i = 0
for comb in combinations(candidatedCoupons, num_coupons):
    couponsPowerset.append((i, list(comb)))
    i += 1

In [7]:
len(couponsPowerset)

258840

In [3]:
for c in candidatedCoupons:
    print(c)

12.31,B000JX5JGI B000R99IFC B000VYIUJS,9.848,B000JX5JGI B000R99IFC B000VYIUJS
17.310000000000002,B000JX5JGI B000R99IFC B000VYIUJS,13.848000000000003,B000JX5JGI B000R99IFC B000VYIUJS
22.310000000000002,B000JX5JGI B000R99IFC B000VYIUJS,17.848000000000003,B000JX5JGI B000R99IFC B000VYIUJS
27.310000000000006,B000JX5JGI B000R99IFC B000VYIUJS,21.848000000000006,B000JX5JGI B000R99IFC B000VYIUJS
32.31000000000001,B000JX5JGI B000R99IFC B000VYIUJS,25.84800000000001,B000JX5JGI B000R99IFC B000VYIUJS
37.31000000000001,B000JX5JGI B000R99IFC B000VYIUJS,29.84800000000001,B000JX5JGI B000R99IFC B000VYIUJS
42.31000000000001,B000JX5JGI B000R99IFC B000VYIUJS,33.848000000000006,B000JX5JGI B000R99IFC B000VYIUJS
47.31000000000002,B000JX5JGI B000R99IFC B000VYIUJS,37.84800000000001,B000JX5JGI B000R99IFC B000VYIUJS


In [5]:
str(candidatedCoupons[0])

'18.99,B000JX5JGI,5.0,B000JX5JGI'

In [14]:
NUM = 16
for i in range(min(NUM, len(candidatedCoupons))):
    print(candidatedCoupons[NUM*1 + i])

12.31,B000VYIUJS,15.0,B000JX5JGI B000R99IFC B000VYIUJS
12.31,B000VYIUJS,25.0,B000JX5JGI B000R99IFC B000VYIUJS
12.31,B000VYIUJS,35.0,B000JX5JGI B000R99IFC B000VYIUJS
12.31,B000VYIUJS,45.0,B000JX5JGI B000R99IFC B000VYIUJS
17.95,B000R99IFC,5.0,B000VYIUJS
17.95,B000R99IFC,5.0,B000R99IFC
17.95,B000R99IFC,15.0,B000R99IFC
17.95,B000R99IFC,5.0,B000JX5JGI
17.95,B000R99IFC,15.0,B000JX5JGI
17.95,B000R99IFC,5.0,B000R99IFC B000VYIUJS
17.95,B000R99IFC,15.0,B000R99IFC B000VYIUJS
17.95,B000R99IFC,25.0,B000R99IFC B000VYIUJS
17.95,B000R99IFC,5.0,B000JX5JGI B000VYIUJS
17.95,B000R99IFC,15.0,B000JX5JGI B000VYIUJS
17.95,B000R99IFC,25.0,B000JX5JGI B000VYIUJS
17.95,B000R99IFC,5.0,B000JX5JGI B000R99IFC


In [14]:
import requests
import json

url = 'http://127.0.0.1:5000/monte-carlo'

params = {
    "coupons_collection":[
        [{"accThreshold": 17.95,
         "accItemset": "B000R99IFC",
         "discount": 17.52,
         "disItemset": "B000JX5JGI"},]
    ]
    }
response = requests.post(url, json=params)
response.json()

{'results': [{'active_nodes': 17.7318,
   'expected_active_nodes': 11.592750668449108,
   'expected_revenue': 218.38104584224516,
   'revenue': 355.3439769999867}]}